In [1]:
import os

os.getcwd()

'/users/eleves-b/2024/mattia.martino/adaptive-quant-trading/scripts'

In [2]:
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt

# Load your OHLC data (Example CSV file)
data = pd.read_csv("../data/INTC_1Min_2023-08-01_2025-02-01.csv", parse_dates=["timestamp"], index_col="timestamp")


In [3]:
data.head()

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2023-08-01 08:01:00+00:00,35.80,35.80,35.80,35.80,1044.0,19.0,35.80
2023-08-01 08:05:00+00:00,35.79,35.79,35.79,35.79,550.0,7.0,35.79
2023-08-01 08:07:00+00:00,35.75,35.75,35.75,35.75,262.0,11.0,35.75
2023-08-01 08:14:00+00:00,35.79,35.79,35.79,35.79,680.0,6.0,35.79
2023-08-01 08:16:00+00:00,35.75,35.75,35.75,35.75,149.0,3.0,35.75


In [4]:
# Define look-back period (commonly between 3-10 days)
n = 60

# Define the function to compute Dual Thrust levels
def dual_thrust_strategy(data, k1, k2, n):
    """Computes buy and sell levels based on Dual Thrust strategy."""
    hh = data["high"].rolling(n).max()
    lc = data["close"].rolling(n).min()
    hc = data["close"].rolling(n).max()
    ll = data["low"].rolling(n).min()
    
    range_val = np.maximum(hh - lc, hc - ll)
    
    buy_line = data["open"] + k1 * range_val
    sell_line = data["open"] - k2 * range_val
    
    return buy_line, sell_line


In [5]:

# Backtest the strategy
def backtest(data, k1, k2, n, initial_balance=10000):
    """Backtest the strategy and calculate returns."""
    buy_line, sell_line = dual_thrust_strategy(data, k1, k2, n)
    balance = initial_balance
    position = 0
    pnl = []

    for i in range(n, len(data)):  # Start after n periods
        if data["close"].iloc[i] > buy_line.iloc[i]:
            position = 1  # Long position
        elif data["close"].iloc[i] < sell_line.iloc[i]:
            position = -1  # Short position
        else:
            position = position  # Hold
        
        daily_return = position * (data["close"].iloc[i] - data["close"].iloc[i - 1])
        pnl.append(daily_return)
    
    returns = np.array(pnl)
    sharpe_ratio = np.mean(returns) / (np.std(returns) + 1e-8)  # Avoid division by zero
    
    return sharpe_ratio, np.sum(returns), np.min(np.cumsum(returns))


In [ ]:
from tqdm import tqdm
# Grid Search Optimization for K1, K2
k1_values = np.arange(0.3, 1.2, 0.1)  # Search between 0.3 and 1.1
k2_values = np.arange(0.3, 1.2, 0.1)
best_k1, best_k2 = 0, 0
best_sharpe = -np.inf

results = []

for k1, k2 in tqdm(itertools.product(k1_values, k2_values)):
    sharpe, total_return, max_drawdown = backtest(data, k1, k2, n)
    results.append((k1, k2, sharpe, total_return, max_drawdown))
    if sharpe > best_sharpe:
        best_sharpe = sharpe
        best_k1, best_k2 = k1, k2

# Convert results to a DataFrame
results_df = pd.DataFrame(results, columns=["K1", "K2", "Sharpe", "Total Return", "Max Drawdown"])


81it [10:51,  8.04s/it]


ModuleNotFoundError: No module named 'ace_tools'

In [16]:
%pip install bayesian-optimization

from bayes_opt import BayesianOptimization

# Define function for Bayesian optimization
def optimize_k1_k2(k1, k2):
    sharpe, _, _ = backtest(data, k1, k2, n)
    return sharpe  # Objective: Maximize Sharpe Ratio

# Define search space
bounds = {"k1": (0.3, 1.2), "k2": (0.3, 1.2)}

# Perform Bayesian Optimization
optimizer = BayesianOptimization(f=optimize_k1_k2, pbounds=bounds, random_state=42)
optimizer.maximize(init_points=5, n_iter=25)

best_k1 = optimizer.max["params"]["k1"]
best_k2 = optimizer.max["params"]["k2"]

print(f"Optimal K1: {best_k1}, Optimal K2: {best_k2}")

  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 52.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 55.7 MB/s eta 0:00:00:00:0100:01
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.
|   iter    |  target   |    k1     |    k2     |
-------------------------------------------------
| 1         | -0.001466 | 0.6371    | 1.156     |
| 2         | 0.002818  | 0.9588    | 0.8388    |
| 3         | 0.0132    | 0.4404    | 0.4404    |
| 4         | -0.001439 | 0.3523    | 1.08      |
| 5         | -0.001376 | 0.841     | 0.9373    |
| 6         | 0.0003268 | 0.9057    | 0.7777    |
| 7         | 0.02676   | 0.3459    | 0.3045    |
| 8         | 0.02819   | 0.3014    | 0.3233    |
| 9         | 0.0